In [10]:
import numpy as np
import deeplake
from matplotlib import pyplot as plt 
from PIL import Image
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader

In [ ]:
# for i in range(10000):
#     mask = Image.open(f"data/masks/mask_{i}.png")
#     mask = np.array(mask, dtype="int")
#     np.save(f"data/masks_npy/mask_{i}.npy", mask)

# mask = Image.open("data/masks/mask_test.png")
# mask = np.array(mask, dtype="int")
# np.save(f"data/masks_npy/mask_test.npy", mask)

In [3]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from preprocessing.utils import MaskedDataset

In [4]:
dataDir = '../data'
masks_folder_path = 'random_npy'

In [5]:
ds = deeplake.load("hub://activeloop/places205")

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/places205



-

hub://activeloop/places205 loaded successfully.



In [6]:
mask_files = os.listdir(os.path.join(dataDir, 'masks', masks_folder_path))
# Create a numpy array to store the masks
masks = np.zeros((len(mask_files), 256, 256))
# Loop over the mask files and load them into the numpy array
for i, mask_file in enumerate(mask_files):
    mask = np.load(os.path.join(dataDir, 'masks', masks_folder_path, mask_file))
    masks[i] = mask
print(masks.shape)

(10001, 256, 256)


In [12]:
cuda = True if torch.cuda.is_available() else False

In [14]:
batch_size = 128 # Batch size during training
image_size = 256 # Spatial size of training images
nc = 3 # Number of channels in the training images. For color images this is 3
nz = 100 # Size of z latent vector (i.e. size of generator input)
ngf = 64 # Size of feature maps in generator
ndf = 64 # Size of feature maps in discriminator
epochs = 5 # Number of training epochs
lr = 0.0002 # Learning rate for optimizers
beta1 = 0.5 # Beta1 hyperparameter for Adam optimizers
ngpu = 1 if cuda else 0 # Number of GPUs available. Use 0 for CPU mode.

In [16]:
# custom weights initialization called on ``netG`` and ``netD``
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [17]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [180]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()

        self.ngpu = ngpu
        self.conv = nn.Sequential(
            # Encoder
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            # Decoder
            nn.ConvTranspose2d(in_channels=512, out_channels=1024, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(in_channels=1024, out_channels=1024, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(in_channels=128, out_channels=3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
            
        )

    def forward(self, input):
        return self.conv(input)
    
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=512, out_channels=1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, input):
        return self.conv(input)

In [7]:
m_ds = MaskedDataset(ds=ds, mask_array= masks)

dataloader_pytorch = DataLoader(m_ds, batch_size = 2, num_workers = 1, shuffle = False)

In [ ]:
for i, sample in enumerate(dataloader_pytorch):
    if i == 0:
        print("images", sample['images'].shape)
        print("masked_images", sample['masked_images'].shape)
        print("labels", sample['labels'].shape)
        for j in range(2):
            plt.imshow(sample['images'][j])
            plt.show()
            plt.imshow(sample['masked_images'][j])
            plt.show()
        # print(sample)
    break

In [125]:
sample.keys()

dict_keys(['images', 'masked_images', 'masks', 'labels'])

In [104]:
x = sample['images']
x = torch.permute(x, (0, 3, 1, 2)).type(torch.FloatTensor)
z = torch.FloatTensor(np.random.normal(0, 1, (x.shape[0], nz, 1, 1)))

In [181]:
generator = Generator(ngpu).to(device)
discriminator = Discriminator(ngpu).to(device)

In [182]:
out_g = generator(x)
out_d = discriminator(out_g)

In [183]:
out_g.shape

torch.Size([2, 3, 256, 256])

In [184]:
out_d.shape

torch.Size([2, 1, 1, 1])